# Assignment Submission for FMUP
## Kishlaya Jaiswal
### Chennai Mathematical Institute - MCS201909
---


# Solution 1

I have choosen the following stocks from Nifty50:
- Kotak Mahindra Bank Ltd (KOTAKBANK)
- Hindustan Unilever Ltd (HINDUNILVR)
- Nestle India Limited (NESTLEIND)


Note:
- I am doing these computations on Apr 2, 2021, and hence using the closing price for this day as my strike price.
- I am using the historical data for the month of February to find the volatility of each of these stocks (volatility computation is done at the end)



In [1]:
import QuantLib as ql


# function to find the price and greeks for a given option
# with it's strike/spot price and it's volatility

def find_price_greeks(spot_price, strike_price, volatility, option_type):
    # construct the European Option
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    exercise = ql.EuropeanExercise(maturity_date)
    european_option = ql.VanillaOption(payoff, exercise)

    # quote the spot price
    spot_handle = ql.QuoteHandle(
        ql.SimpleQuote(spot_price)
    )
    flat_ts = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, risk_free_rate, day_count)
    )
    dividend_yield = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, dividend_rate, day_count)
    )
    flat_vol_ts = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
    )
    
    # create the Black Scholes process
    bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                           dividend_yield, 
                                           flat_ts, 
                                           flat_vol_ts)

    # set the engine to use the above process
    european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
    
    return european_option


tickers = ["KOTAKBANK", "HINDUNILVR", "NESTLEIND"]

# spot price = closing price as on Mar 1, 2021
spot = {"KOTAKBANK":1845.35, 
        "HINDUNILVR":2144.70, 
        "NESTLEIND":16288.20}

# strike price = closing price as on Apr 2, 2021
strike = {"KOTAKBANK":1804.45, 
          "HINDUNILVR":2399.45, 
          "NESTLEIND":17102.15}

# historical volatility from the past month's data
vol = {"KOTAKBANK":0.38, 
       "HINDUNILVR":0.15, 
       "NESTLEIND":0.18}

# date of option purchase
calculation_date = ql.Date(1,3,2021)

# exercise date
# this excludes the holidays in the Indian calendar
calendar = ql.India()
period = ql.Period(65, ql.Days)
maturity_date = calendar.advance(calculation_date, period)

# rate of interest
risk_free_rate = 0.06

# other settings
dividend_rate =  0.0
day_count = ql.Actual365Fixed()
ql.Settings.instance().evaluationDate = calculation_date

# store final variables for future calculations
delta = {}
gamma = {}
vega = {}

# print settings
format_type_head = "{:<15}" + ("{:<12}" * 7)
format_type = "{:<15}{:<12}" + ("{:<12.2f}" * 6)
print(format_type_head.format("Name", "Type", "Price", "Delta", "Gamma", "Rho", "Theta", "Vega"))
print()

for ticker in tickers:
    option = find_price_greeks(spot[ticker], strike[ticker], vol[ticker], ql.Option.Call)
    print(format_type.format(ticker, "Call", option.NPV(), 
                             option.delta(), option.gamma(), 
                             option.rho(), option.theta(), option.vega()))
    
    delta[ticker]  = option.delta()
    gamma[ticker]  = option.gamma()
    vega[ticker]   = option.vega()
    
    option = find_price_greeks(spot[ticker], strike[ticker], vol[ticker], ql.Option.Put)
    print(format_type.format(ticker, "Put", option.NPV(), 
                             option.delta(), option.gamma(), 
                             option.rho(), option.theta(), option.vega()))
    
    print()


Name           Type        Price       Delta       Gamma       Rho         Theta       Vega        

KOTAKBANK      Call        175.18      0.62        0.00        244.54      -323.09     356.05      
KOTAKBANK      Put         106.90      -0.38       0.00        -208.25     -216.47     356.05      

HINDUNILVR     Call        8.08        0.11        0.00        56.41       -72.03      199.56      
HINDUNILVR     Put         226.43      -0.89       0.00        -545.68     69.76       199.56      

NESTLEIND      Call        363.26      0.37        0.00        1456.88     -1442.77    3113.31     
NESTLEIND      Put         917.75      -0.63       0.00        -2834.54    -432.21     3113.31     



### Delta Gamma Vega neutrality

First we make the Gamma and Vega neutral by taking 
- x units of KOTAKBANK
- y units of HINDUNILVR
- 1 unit of NESTLEIND

To solve for x,y we have the following:

In [2]:
import numpy as np

G1, G2, G3 = gamma["KOTAKBANK"], gamma["HINDUNILVR"], gamma["NESTLEIND"]
V1, V2, V3 = vega["KOTAKBANK"], vega["HINDUNILVR"], vega["NESTLEIND"]

# Solve the following equation:
# G1 x + G2 y + G3 = 0
# V1 x + V2 y + V3 = 0

A = np.array([[G1, G2], [V1, V2]])
b = np.array([-G3, -V3])
z = np.linalg.solve(A, b)

print("x = {:.2f}".format(z[0]))
print("y = {:.2f}".format(z[1]))
print()

final_delta = z[0]*delta["KOTAKBANK"] + z[1]*delta["HINDUNILVR"] + delta["NESTLEIND"]
print("Delta of portfolio is {:.2f}".format(final_delta))

x = -18.46
y = 17.34

Delta of portfolio is -9.13


## Final Strategy

- Take a short position of 18.46 units of Kotak Mahindra Bank Ltd Call Option
- Take a long position of 17.34 units of Hindustan Unilever Ltd Call Option
- Take a long position of 1 unit of Nestle India Limited Call Option
- Take a long position of 9.13 units of Nestle India Limited Stock

This will yield a portfolio with Delta, Gamma and Vega neutral.

# Solution 2

Using Taylor expansion, we get
$$\Delta P = \frac{\partial P}{\partial y} \Delta y + \frac12 \frac{\partial^2 P}{\partial y^2}(\Delta y)^2$$
$$\implies \frac{\Delta P}{P} = -D \Delta y + \frac12 C (\Delta y)^2$$

where $D$ denotes duration and $C$ denotes convexity of a bond.

We remark that the duration of the bonds we are comparing are same and fixed.

---

<p>With that being said, let's say the interest rates fall, then we have $$\Delta y < 0 \implies - D \Delta y + C \Delta y^2 > 0 \implies \Delta P > 0$$
Now for the bond with greater convexity, $C \Delta y^2$ has a large value hence $\Delta P$ has to be large and so we get that "Greater convexity translates into greater price gains as interest rates fall"
</p>

---

Now suppose interest rates rise that is $\Delta y > 0$, then we $-D \Delta y < 0$, that is the price of the bonds decreases but the bond with greater convexity will add up for a large $C \Delta y^2$ also and so the price decrease will be less for bond with high convexity.

This explains "Lessened price declines as interest rates rise"

# Solution 3

In [3]:
import QuantLib as ql

# function to calculate coupon value
def find_coupon(pv, r, m, n):
    discount_factor = (r/m) / (1 - (1 + r/m)**(-n*m))
    C = pv * discount_factor
    return C

# loan settings
loan_amt = 0.8*1000000
rate = 0.12
pay = find_coupon(loan_amt, rate, 12, 5)
month = ql.Date(15,8,2021)
period = ql.Period('1m')

# print settings
print("Monthly coupon is: {:.2f}".format(pay))
print()
format_type = "{:<15}" * 4
print(format_type.format("Date", "Interest", "Principal", "Remaining"))


while loan_amt > 0:
    interest = loan_amt * rate / 12
    principal = pay - interest
    loan_amt = loan_amt - principal
    print(format_type.format(month.ISO(), "{:.2f}".format(interest), "{:.2f}".format(principal), "{:.2f}".format(loan_amt)))

    if round(loan_amt) == 0:
        break
    month = month + period

Monthly coupon is: 17795.56

Date           Interest       Principal      Remaining      
2021-08-15     8000.00        9795.56        790204.44      
2021-09-15     7902.04        9893.51        780310.93      
2021-10-15     7803.11        9992.45        770318.48      
2021-11-15     7703.18        10092.37       760226.11      
2021-12-15     7602.26        10193.30       750032.81      
2022-01-15     7500.33        10295.23       739737.58      
2022-02-15     7397.38        10398.18       729339.40      
2022-03-15     7293.39        10502.16       718837.23      
2022-04-15     7188.37        10607.19       708230.05      
2022-05-15     7082.30        10713.26       697516.79      
2022-06-15     6975.17        10820.39       686696.40      
2022-07-15     6866.96        10928.59       675767.80      
2022-08-15     6757.68        11037.88       664729.92      
2022-09-15     6647.30        11148.26       653581.67      
2022-10-15     6535.82        11259.74       642321.92  

### Volatility Computation for Problem 1

In [4]:
import math

def get_volatility(csv):
    data = csv.split('\n')[1:]
    data = map(lambda x: x.split(','), data)
    closing_prices = list(map(lambda x: float(x[-2]), data))
    
    n = len(closing_prices)
    
    log_returns = []
    for i in range(1,n):
        log_returns.append(math.log(closing_prices[i]/closing_prices[i-1]))
    
    mu = sum(log_returns)/(n-1)
    
    tmp = map(lambda x: (x-mu)**2, log_returns)
    
    vol = math.sqrt(sum(tmp)/(n-1)) * math.sqrt(252)
    return vol

kotak_csv = '''Date,Open,High,Low,Close,Adj Close,Volume
2021-02-01,1730.000000,1810.000000,1696.250000,1801.349976,1801.349976,220763
2021-02-02,1825.000000,1878.650024,1801.349976,1863.500000,1863.500000,337556
2021-02-03,1875.000000,1882.349976,1820.099976,1851.849976,1851.849976,147146
2021-02-04,1857.900024,1914.500000,1831.050049,1911.250000,1911.250000,188844
2021-02-05,1921.000000,1997.900024,1915.000000,1982.550049,1982.550049,786773
2021-02-08,1995.000000,2029.949951,1951.949951,1956.300049,1956.300049,212114
2021-02-09,1950.000000,1975.000000,1938.000000,1949.199951,1949.199951,62613
2021-02-10,1954.550049,1961.849976,1936.300049,1953.650024,1953.650024,143830
2021-02-11,1936.000000,1984.300049,1936.000000,1961.300049,1961.300049,120121
2021-02-12,1966.000000,1974.550049,1945.599976,1951.449951,1951.449951,86860
2021-02-15,1954.000000,1999.000000,1954.000000,1986.199951,1986.199951,135074
2021-02-16,1995.000000,2048.949951,1995.000000,2021.650024,2021.650024,261589
2021-02-17,2008.500000,2022.400024,1969.500000,1989.150024,1989.150024,450365
2021-02-18,1980.000000,1982.349976,1938.000000,1945.300049,1945.300049,193234
2021-02-19,1945.000000,1969.599976,1925.050049,1937.300049,1937.300049,49189
2021-02-22,1941.000000,1961.650024,1921.650024,1948.550049,1948.550049,44651
2021-02-23,1955.000000,1961.900024,1867.000000,1873.150024,1873.150024,118138
2021-02-24,1875.199951,1953.949951,1852.000000,1919.000000,1919.000000,454695
2021-02-25,1935.000000,1964.949951,1886.900024,1895.349976,1895.349976,195212
2021-02-26,1863.000000,1868.000000,1773.099976,1782.349976,1782.349976,180729'''

hind_csv = '''Date,Open,High,Low,Close,Adj Close,Volume
2021-02-01,2265.000000,2286.000000,2226.550049,2249.149902,2249.149902,130497
2021-02-02,2271.000000,2275.000000,2207.699951,2231.850098,2231.850098,327563
2021-02-03,2234.000000,2256.699951,2218.199951,2232.600098,2232.600098,121232
2021-02-04,2234.000000,2258.449951,2226.949951,2247.050049,2247.050049,533609
2021-02-05,2252.000000,2285.000000,2241.000000,2270.350098,2270.350098,254911
2021-02-08,2275.000000,2287.000000,2233.000000,2237.800049,2237.800049,211465
2021-02-09,2247.000000,2254.000000,2211.199951,2216.649902,2216.649902,171285
2021-02-10,2216.649902,2240.000000,2213.449951,2235.899902,2235.899902,185915
2021-02-11,2245.000000,2267.500000,2235.000000,2262.399902,2262.399902,121168
2021-02-12,2270.000000,2270.649902,2232.199951,2241.899902,2241.899902,33016
2021-02-15,2252.000000,2261.500000,2212.100098,2215.850098,2215.850098,91240
2021-02-16,2225.000000,2228.399902,2190.500000,2196.899902,2196.899902,101652
2021-02-17,2191.000000,2200.000000,2160.300049,2164.649902,2164.649902,138504
2021-02-18,2165.000000,2168.449951,2143.050049,2147.750000,2147.750000,110272
2021-02-19,2150.000000,2193.649902,2148.000000,2181.149902,2181.149902,150398
2021-02-22,2200.000000,2201.699951,2161.100098,2167.250000,2167.250000,98782
2021-02-23,2173.550049,2192.000000,2169.399902,2177.949951,2177.949951,22743
2021-02-24,2179.000000,2183.949951,2104.250000,2181.600098,2181.600098,329265
2021-02-25,2190.000000,2190.000000,2160.000000,2163.600098,2163.600098,357853
2021-02-26,2151.149902,2182.000000,2122.000000,2132.050049,2132.050049,158925'''

nestle_csv = '''Date,Open,High,Low,Close,Adj Close,Volume
2021-02-01,17162.099609,17277.000000,16996.449219,17096.949219,17096.949219,3169
2021-02-02,17211.000000,17328.099609,16800.000000,17189.349609,17189.349609,3852
2021-02-03,17247.449219,17284.000000,17064.349609,17155.400391,17155.400391,2270
2021-02-04,17250.000000,17250.000000,17054.800781,17073.199219,17073.199219,13193
2021-02-05,17244.000000,17244.000000,17019.949219,17123.300781,17123.300781,2503
2021-02-08,17199.949219,17280.000000,17107.349609,17213.550781,17213.550781,7122
2021-02-09,17340.000000,17510.699219,17164.050781,17325.800781,17325.800781,2714
2021-02-10,17396.900391,17439.300781,17083.800781,17167.699219,17167.699219,3341
2021-02-11,17167.699219,17442.000000,17165.550781,17416.650391,17416.650391,2025
2021-02-12,17449.849609,17500.000000,17241.000000,17286.099609,17286.099609,3486
2021-02-15,17290.000000,17500.000000,17280.000000,17484.500000,17484.500000,1927
2021-02-16,17600.000000,17634.599609,17141.250000,17222.449219,17222.449219,7901
2021-02-17,16900.000000,16900.000000,16360.000000,16739.900391,16739.900391,28701
2021-02-18,17050.000000,17050.000000,16307.000000,16374.150391,16374.150391,13711
2021-02-19,16395.000000,16477.599609,16214.450195,16386.099609,16386.099609,5777
2021-02-22,16400.000000,16531.050781,16024.599609,16099.200195,16099.200195,9051
2021-02-23,16123.000000,16250.000000,16003.000000,16165.250000,16165.250000,6261
2021-02-24,16249.000000,16800.000000,15900.000000,16369.950195,16369.950195,18003
2021-02-25,16394.699219,16394.699219,16102.000000,16114.349609,16114.349609,18735
2021-02-26,16075.000000,16287.200195,16010.000000,16097.700195,16097.700195,13733'''

print("Annualized Volatility of KOTAKBANK  is {:.2f}%".format(get_volatility(kotak_csv)*100))
print("Annualized Volatility of HINDUNILVR is {:.2f}%".format(get_volatility(hind_csv)*100))
print("Annualized Volatility of NESTLEIND  is {:.2f}%".format(get_volatility(nestle_csv)*100))

Annualized Volatility of KOTAKBANK  is 38.66%
Annualized Volatility of HINDUNILVR is 15.16%
Annualized Volatility of NESTLEIND  is 18.86%
